## working

In [ ]:
import h5py
import cupy as cp
import napari
import tifffile

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"

# Load the TIFF image (shape: T, Z, C, Y, X)
full_image_data = tifffile.imread(tiff_path)

image_data = full_image_data.copy()
num_timepoints, num_z, _, num_y, num_x = image_data.shape  # Extract image dimensions

# Voxel spacing for correct 3D rendering
z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104
spacing = [z_spacing, y_spacing, x_spacing]

# Initialize an empty CuPy volume for kinetochores (T, Z, Y, X)
kinetochore_volume = cp.zeros((num_timepoints, num_z, num_y, num_x), dtype=cp.uint8)

# Define kinetochore sphere radius
sphere_radius = 2  # Adjust as needed

# Function to create a sphere using CuPy
def create_sphere(center, radius, shape):
    """Generates a binary sphere in a 3D CuPy array at the specified center."""
    z, y, x = cp.indices(shape, dtype=cp.float32)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image using CuPy
def generate_mask(image_3d, Z_coords, Y_coords, X_coords, spot_radius=2):
    """Generates a binary mask with spheres at kinetochore locations using CuPy."""
    
    # Create an empty CuPy mask
    mask = cp.zeros_like(image_3d, dtype=cp.uint8)

    # Add spheres to the mask at detected spot locations
    for z, y, x in zip(Z_coords, Y_coords, X_coords):
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d.shape)
        mask[sphere_mask] = 1

    return mask

# Load kinetochore coordinates for all timepoints using CuPy
with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]  # Shape (T, 1)

                # Loop through all timepoints
                for T_index in range(num_timepoints):
                    first_entry_ref = allCoordPix[T_index, 0]

                    if isinstance(first_entry_ref, h5py.Reference):
                        first_data = f[first_entry_ref]
                        array_data = cp.array(first_data[...])  # Load directly into CuPy
                        print(f"T={T_index}: Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

                        # Extract ZYX coordinates and convert MATLAB (1-based) to Python (0-based) indexing
                        Z_coords = cp.round(array_data[2] - 1).astype(cp.int32)  # Z values
                        Y_coords = cp.round(array_data[0] - 1).astype(cp.int32)  # Y values
                        X_coords = cp.round(array_data[1] - 1).astype(cp.int32)  # X values

                        # Ensure coordinates are within bounds
                        valid_mask = (Z_coords >= 0) & (Z_coords < num_z) & \
                                     (Y_coords >= 0) & (Y_coords < num_y) & \
                                     (X_coords >= 0) & (X_coords < num_x)
                        Z_coords, Y_coords, X_coords = Z_coords[valid_mask], Y_coords[valid_mask], X_coords[valid_mask]

                        # Generate mask using CuPy
                        kinetochore_volume[T_index] = generate_mask(kinetochore_volume[T_index], Z_coords, Y_coords, X_coords)

# Transfer CuPy array back to NumPy for visualization in Napari
kinetochore_volume_np = cp.asnumpy(kinetochore_volume)

# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add the entire image time series (T, Z, C, Y, X)
viewer.add_image(image_data[:, :, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='attenuated_mip', opacity=0.5)
viewer.add_image(image_data[:, :, 1], colormap='cyan', scale=spacing, name='Channel 2', rendering='attenuated_mip', opacity=0.5)

# Add kinetochore spheres as a 4D volume
viewer.add_labels(kinetochore_volume_np, scale=spacing, name="Kinetochores", opacity=0.8)

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles for 3D view

# Run Napari
napari.run()


## With offset

In [6]:
import h5py
import cupy as cp
import napari
import tifffile

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"

# Load the TIFF image (shape: T, Z, C, Y, X)
full_image_data = tifffile.imread(tiff_path)

# Convert to CuPy for fast processing
image_data = cp.asarray(full_image_data.copy())

num_timepoints, num_z, _, num_y, num_x = image_data.shape  # Extract image dimensions

# Define voxel spacing for correct 3D rendering
z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104
spacing = [z_spacing, y_spacing, x_spacing]

# Define the offset for Channel 2 (Z, Y, X)
channel_2_offset = (0, 1, 2)  # Shift: Z=0, Y=2, X=1

# Store the original Channel 2 before shifting
original_channel_2 = image_data[:, :, 1, :, :].copy()

# Apply offset to Channel 2 using `cp.roll()`
image_data[:, :, 1, :, :] = cp.roll(image_data[:, :, 1, :, :], shift=channel_2_offset, axis=(1, 2, 3))

# Initialize an empty CuPy volume for kinetochores (T, Z, Y, X)
kinetochore_volume = cp.zeros((num_timepoints, num_z, num_y, num_x), dtype=cp.uint8)

# Define kinetochore sphere radius
sphere_radius = 2  # Adjust as needed

# Function to create a sphere using CuPy
def create_sphere(center, radius, shape):
    """Generates a binary sphere in a 3D CuPy array at the specified center."""
    z, y, x = cp.indices(shape, dtype=cp.float32)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image using CuPy
def generate_mask(image_3d, Z_coords, Y_coords, X_coords, spot_radius=2):
    """Generates a binary mask with spheres at kinetochore locations using CuPy."""
    
    # Create an empty CuPy mask
    mask = cp.zeros_like(image_3d, dtype=cp.uint8)

    # Add spheres to the mask at detected spot locations
    for z, y, x in zip(Z_coords, Y_coords, X_coords):
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d.shape)
        mask[sphere_mask] = 1

    return mask

# Load kinetochore coordinates for all timepoints using CuPy
with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]  # Shape (T, 1)

                # Loop through all timepoints
                for T_index in range(num_timepoints):
                    first_entry_ref = allCoordPix[T_index, 0]

                    if isinstance(first_entry_ref, h5py.Reference):
                        first_data = f[first_entry_ref]
                        array_data = cp.array(first_data[...])  # Load directly into CuPy
                        print(f"T={T_index}: Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

                        # Extract ZYX coordinates and convert MATLAB (1-based) to Python (0-based) indexing
                        Z_coords = cp.round(array_data[2] - 1).astype(cp.int32)  # Z values
                        Y_coords = cp.round(array_data[0] - 1).astype(cp.int32)  # Y values
                        X_coords = cp.round(array_data[1] - 1).astype(cp.int32)  # X values

                        # Ensure coordinates are within bounds
                        valid_mask = (Z_coords >= 0) & (Z_coords < num_z) & \
                                     (Y_coords >= 0) & (Y_coords < num_y) & \
                                     (X_coords >= 0) & (X_coords < num_x)
                        Z_coords, Y_coords, X_coords = Z_coords[valid_mask], Y_coords[valid_mask], X_coords[valid_mask]

                        # Generate mask using CuPy
                        kinetochore_volume[T_index] = generate_mask(kinetochore_volume[T_index], Z_coords, Y_coords, X_coords)

# Transfer CuPy array back to NumPy for visualization in Napari
kinetochore_volume_np = cp.asnumpy(kinetochore_volume)
image_data_np = cp.asnumpy(image_data)
original_channel_2_np = cp.asnumpy(original_channel_2)  # Convert original Channel 2 back to NumPy

# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add the entire image time series (T, Z, C, Y, X)
viewer.add_image(image_data_np[:, :, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='attenuated_mip', opacity=0.5)
viewer.add_image(original_channel_2_np, colormap='cyan', scale=spacing, name='Channel 2 (Original)', rendering='attenuated_mip', opacity=0.5)
viewer.add_image(image_data_np[:, :, 1], colormap='yellow', scale=spacing, name='Channel 2 (Shifted)', rendering='attenuated_mip', opacity=0.5)

# Add kinetochore spheres as a 4D volume
viewer.add_labels(kinetochore_volume_np, scale=spacing, name="Kinetochores", opacity=0.8)

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles for 3D view

# Run Napari
napari.run()


T=0: Extracted array_data shape: (6, 125), Type: float64
T=1: Extracted array_data shape: (6, 59), Type: float64
T=2: Extracted array_data shape: (6, 86), Type: float64
T=3: Extracted array_data shape: (6, 89), Type: float64
T=4: Extracted array_data shape: (6, 94), Type: float64
T=5: Extracted array_data shape: (6, 76), Type: float64
T=6: Extracted array_data shape: (6, 127), Type: float64
T=7: Extracted array_data shape: (6, 70), Type: float64
T=8: Extracted array_data shape: (6, 86), Type: float64
T=9: Extracted array_data shape: (6, 68), Type: float64
T=10: Extracted array_data shape: (6, 90), Type: float64
T=11: Extracted array_data shape: (6, 98), Type: float64
T=12: Extracted array_data shape: (6, 93), Type: float64
T=13: Extracted array_data shape: (6, 81), Type: float64
T=14: Extracted array_data shape: (6, 118), Type: float64
T=15: Extracted array_data shape: (6, 110), Type: float64
T=16: Extracted array_data shape: (6, 65), Type: float64
T=17: Extracted array_data shape: (6,

In [26]:
import h5py
import numpy as np

file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"

with h5py.File(file_path, 'r') as f:
    # Dereference the first entry in dataStruct
    data_struct = f["dataStruct"][0, 0]  # Assuming first struct is relevant
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Dereference dataStruct
        
        # Navigate to initCoord
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            # Check if allCoordPix exists
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]
                
                # Read the first entry
                first_entry_ref = allCoordPix[40, 0]  # Assuming shape (127,1)
                
                if isinstance(first_entry_ref, h5py.Reference):
                    first_data = f[first_entry_ref]  # Dereference
                    print(f"First entry shape: {first_data.shape}, Type: {first_data.dtype}")
                    
                    # Load into a NumPy array
                    array_data = first_data[...]
                    print("First entry array:", array_data)
                else:
                    print("First entry is not a reference.")
            else:
                print("allCoordPix not found in initCoord.")
        


First entry shape: (6, 80), Type: float64
First entry array: [[1.34960881e+02 1.55093485e+02 1.36708452e+02 1.21191147e+02
  1.69424211e+02 1.19061538e+02 1.63067673e+02 1.30501061e+02
  1.41384288e+02 1.73764484e+02 1.60033560e+02 1.29842712e+02
  1.51080964e+02 1.60610009e+02 1.27493171e+02 1.04850252e+02
  1.61826521e+02 1.54792151e+02 1.47254149e+02 1.22453061e+02
  1.62688666e+02 1.52712275e+02 1.18902545e+02 1.00593792e+02
  1.36280828e+02 1.44088539e+02 1.41283013e+02 1.48143459e+02
  1.24475584e+02 1.74181332e+02 9.62688667e+01 1.59040597e+02
  1.43855369e+02 1.41078916e+02 1.14456612e+02 1.62011528e+02
  1.47916207e+02 1.09320316e+02 1.12435571e+02 1.67594002e+02
  1.53050876e+02 1.70891432e+02 1.08171164e+02 1.05017496e+02
  1.67177757e+02 9.99671086e+01 1.41017282e+02 1.32063578e+02
  8.50056133e+01 1.30459260e+02 1.52103305e+02 1.40083221e+02
  1.24628095e+02 1.69943342e+02 1.17157924e+02 1.58693400e+02
  1.25235371e+02 1.37475197e+02 1.62336385e+02 1.25875072e+02
  1.49649

In [1]:
import tifffile
data = tifffile.imread(r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif")

In [2]:
data.shape

(127, 60, 2, 255, 496)

In [20]:
ls Z:\Shared299\KinetochoreClub\"for Nina"\"03 PhD lattice movies"\"20241107 NP_002 MC256 2-colour 4-sec NEBD"\"Decon 10 iter with day-old PSFs"\"Cell033_Slide 6 capture 8_DC_DS.tif"

 Volume in drive Z is SYSTEM
 Volume Serial Number is ECF6-6273

 Directory of Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs

21/01/2025  20:20    <DIR>          .
15/11/2024  15:37    <DIR>          ..
07/11/2024  09:26         9,984,487 488_PSF.tif
07/11/2024  09:26        10,931,287 560_PSF.tif
14/11/2024  09:58     4,288,489,936 Cell032_Slide 3 capture 6_DC_DS.tif
14/11/2024  11:20        68,931,390 Cell032_Slide3capture6_DC_DS_MIP.tif
14/11/2024  10:05     3,879,768,735 Cell033_Slide 6 capture 8_DC_DS.tif
14/11/2024  11:22        64,303,445 Cell033_Slide6capture8_DC_DS_MIP.tif
14/11/2024  10:22    13,992,481,455 Cell034_Slide 7 capture 3_DC_DS.tif
14/11/2024  11:29       223,400,697 Cell034_Slide7capture3_DC_DS_MIP.tif
14/11/2024  09:50    <DIR>          Slide 3 capture 6
14/11/2024  09:17     2,392,956,361 Slide 3 capture 6.sld
21/01/2025  20:21    <DIR>          Slide 6 capture 8
14/11/2024  

In [14]:
ls NP_002\ Tracking

diags_kitjobset_241126_Cell033_withSisterAlignment.txt
diags_kitjobset_241126_Cell037_withSisterAlignment.txt
kitjobset_241126_Cell033_withSisterAlignment.mat
kitjobset_241126_Cell037_withSisterAlignment.mat
kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat
kittracking001-kitjobset_241126_Cell037_withSisterAlignment-Cell037_DC_DS.mat


In [43]:
import h5py
import numpy as np
import napari
import tifffile

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"
T_index = 40

# Load array_data from .mat file
with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]
                
                # Read the 41st entry (index 40)
                first_entry_ref = allCoordPix[T_index, 0]
                
                if isinstance(first_entry_ref, h5py.Reference):
                    first_data = f[first_entry_ref]
                    array_data = first_data[...]  # Extract actual numpy array
                    print(f"Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

# Extract ZYX coordinates
Z_coords = array_data[2]  # Z values
Y_coords = array_data[1]  # Y values
X_coords = array_data[0]  # X values

# Stack into (N, 3) for Napari visualization
coords = np.stack((Z_coords, X_coords, Y_coords), axis=1)

# Load the TIFF image
image_data = tifffile.imread(tiff_path)  # Shape: (127, 60, 2, 255, 496) (T, Z, C, Y, X)

# Extract the required timepoint (T=40)
if T_index >= image_data.shape[0]:
    print(f"Warning: T={T_index} exceeds available range. Using last frame.")
    T_index = image_data.shape[0] - 1  # Use last available frame

image_T = image_data[T_index]  # Extract data for T=40 -> Shape (Z, C, Y, X)

z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104


spacing = [z_spacing, y_spacing, x_spacing]
# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add both fluorescence channels as volumetric images
viewer.add_image(image_T[:, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='attenuated_mip')  # NDC80
viewer.add_image(image_T[:, 1], colormap='cyan', scale=spacing, name='Channel 2', rendering='attenuated_mip')  # Nuclear envelope

# Overlay kinetochore coordinates as spherical markers
viewer.add_points(coords, size=3, scale=spacing, face_color='yellow', edge_color='black', ndim=3, name="Kinetochores")

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles to better visualize the 3D structure

# Run Napari
napari.run()


Extracted array_data shape: (6, 80), Type: float64


E:\Scott\Software\Anaconda\Envs\napari_env\lib\site-packages\napari\utils\migrations.py:101: FutureWarning: Argument 'edge_color' is deprecated, please use 'border_color' instead. The argument 'edge_color' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  return func(*args, **kwargs)


In [41]:
Z_coords

array([43.8096627 , 25.06144907, 17.05881536, 27.71151597, 37.6900739 ,
       45.68530296, 39.08733993, 12.99136826, 44.12937681, 34.58616021,
       25.92322006, 24.11149114, 28.14847298, 37.93403356, 42.72118235,
       25.29178991, 33.26579474, 15.34603977, 33.90016766, 15.14801006,
       30.41514987, 30.69993909, 25.60435223, 31.90871498, 37.71612439,
       34.0536581 , 36.0262594 , 46.84528236, 27.51066104, 18.37793809,
       39.72016586, 27.80963048, 46.06268396, 44.12121589, 37.77462477,
       40.61472298, 36.56000697, 25.96166488, 24.74506979, 36.72033549,
       43.97290913, 28.80498918, 15.40758689, 14.77706889, 22.17465064,
       30.11188558, 28.81895309, 33.54518035, 35.36229254, 42.78387823,
       25.99541186, 18.79478726, 29.5326667 , 35.75888241, 44.82342828,
       39.94621345, 10.54266696, 42.0328896 , 28.61752683, 23.98635535,
       29.50129245, 30.98499267, 43.89779823, 23.28006906, 34.33579125,
       14.69358681, 13.33094938, 31.42786692, 25.11451822, 30.58

In [42]:
X_coords

array([134.96088053, 155.09348501, 136.70845241, 121.19114687,
       169.42421125, 119.06153757, 163.0676729 , 130.50106115,
       141.38428834, 173.76448367, 160.03356016, 129.84271197,
       151.08096394, 160.61000859, 127.49317072, 104.85025247,
       161.82652141, 154.79215108, 147.25414919, 122.45306127,
       162.68866649, 152.71227496, 118.90254546, 100.59379159,
       136.28082825, 144.08853905, 141.2830126 , 148.14345939,
       124.47558354, 174.18133151,  96.26886666, 159.04059722,
       143.85536915, 141.07891641, 114.45661237, 162.01152837,
       147.91620746, 109.32031558, 112.43557128, 167.59400175,
       153.05087579, 170.89143216, 108.17116353, 105.01749601,
       167.17775659,  99.96710862, 141.0172819 , 132.06357755,
        85.0056133 , 130.45925983, 152.10330544, 140.08322074,
       124.62809537, 169.94334211, 117.15792375, 158.69340024,
       125.23537081, 137.47519692, 162.33638484, 125.87507192,
       149.64900655, 162.2200869 , 126.99238024, 109.58

In [40]:
Y_coords

array([247.79057637, 198.89529383, 239.10565358, 241.88342874,
       200.50829522, 203.77334236, 249.95796395, 277.16992046,
       193.38195122, 207.27399371, 224.00626788, 250.66120561,
       226.20769487, 219.4639053 , 228.85984097, 261.62950946,
       259.1454921 , 278.02983592, 225.23518519, 252.80716483,
       210.26284635, 241.92167483, 209.52044672, 208.63971928,
       258.14824554, 230.37048865, 182.94438993, 210.03630945,
       282.48126685, 267.49425985, 239.59314544, 276.96858882,
       215.18513032, 241.92938174, 263.65569384, 233.30360685,
       258.88076212, 241.8103714 , 240.90795767, 221.57272989,
       204.11661233, 261.8102383 , 247.4312356 , 249.88152248,
       249.4641981 , 251.22996158, 249.7064711 , 275.08751988,
       248.84484548, 245.72705546, 200.59092879, 239.64980942,
       239.61152928, 204.58678498, 198.68286027, 250.62142277,
       275.80361032, 195.35131718, 227.8748676 , 253.36187991,
       230.81331601, 216.62228561, 233.84045028, 263.10

In [50]:
import h5py
import numpy as np
import napari
import tifffile
from scipy.ndimage import gaussian_filter

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"

# Load the TIFF image (shape: T, Z, C, Y, X)
image_data = tifffile.imread(tiff_path)
num_timepoints, num_z, _, num_y, num_x = image_data.shape  # Extract image dimensions

print(image_data.shape)
# Voxel spacing for correct 3D rendering
z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104
spacing = [z_spacing, y_spacing, x_spacing]

# Initialize an empty volume for kinetochores (T, Z, Y, X)
kinetochore_volume = np.zeros((num_timepoints, num_z, num_y, num_x), dtype=np.float32)

# Load kinetochore coordinates for all timepoints
sphere_radius = 2  # Radius of the kinetochore spheres in pixels

with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]  # Shape (T, 1)

                # Loop through all timepoints
                for T_index in range(num_timepoints):
                    first_entry_ref = allCoordPix[T_index, 0]

                    if isinstance(first_entry_ref, h5py.Reference):
                        first_data = f[first_entry_ref]
                        array_data = first_data[...]  # Extract actual numpy array
                        print(f"T={T_index}: Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

                        # Extract ZYX coordinates
                        Z_coords = np.round(array_data[2]).astype(int)  # Z values
                        Y_coords = np.round(array_data[1]).astype(int)  # Y values
                        X_coords = np.round(array_data[0]).astype(int)  # X values
                        print(Z_coords.shape, Y_coords.shape, X_coords.shape)

                        # # Ensure coordinates are within bounds
                        # valid_mask = (Z_coords >= 0) & (Z_coords < num_z) & \
                        #              (Y_coords >= 0) & (Y_coords < num_y) & \
                        #              (X_coords >= 0) & (X_coords < num_x)
                        # Z_coords, Y_coords, X_coords = Z_coords[valid_mask], Y_coords[valid_mask], X_coords[valid_mask]

                        # Add spherical representations in the 4D volume
                        for z, y, x in zip(Z_coords, Y_coords, X_coords):
                            kinetochore_volume[T_index, z, y, x] = 1.0  # Binary marking

# Apply Gaussian blur to simulate spheres
kinetochore_volume = gaussian_filter(kinetochore_volume, sigma=(0, sphere_radius, sphere_radius, sphere_radius))

# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add the entire image time series (T, Z, C, Y, X)
viewer.add_image(image_data[:, :, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='attenuated_mip')
viewer.add_image(image_data[:, :, 1], colormap='cyan', scale=spacing, name='Channel 2', rendering='attenuated_mip')

# Add kinetochore spheres as a 4D volume
viewer.add_image(kinetochore_volume, colormap='yellow', scale=spacing, name="Kinetochores", rendering='attenuated_mip', opacity=0.6)

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles for 3D view

# Run Napari
napari.run()


(127, 60, 2, 255, 496)
T=0: Extracted array_data shape: (6, 125), Type: float64
(125,) (125,) (125,)


IndexError: index 257 is out of bounds for axis 2 with size 255

In [1]:
import h5py
import cupy as cp
import napari
import tifffile

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"

# Load the TIFF image (shape: T, Z, C, Y, X)
full_image_data = tifffile.imread(tiff_path)
image_data = full_image_data[0:2]
num_timepoints, num_z, _, num_y, num_x = image_data.shape  # Extract image dimensions

# Voxel spacing for correct 3D rendering
z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104
spacing = [z_spacing, y_spacing, x_spacing]

# Initialize an empty CuPy volume for kinetochores (T, Z, Y, X)
kinetochore_volume = cp.zeros((num_timepoints, num_z, num_y, num_x), dtype=cp.uint8)

# Define kinetochore sphere radius
sphere_radius = 2  # Adjust as needed

# Function to create a sphere using CuPy
def create_sphere(center, radius, shape):
    """Generates a binary sphere in a 3D CuPy array at the specified center."""
    z, y, x = cp.indices(shape, dtype=cp.float32)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image using CuPy
def generate_mask(image_3d, Z_coords, Y_coords, X_coords, spot_radius=2):
    """Generates a binary mask with spheres at kinetochore locations using CuPy."""
    
    # Create an empty CuPy mask
    mask = cp.zeros_like(image_3d, dtype=cp.uint8)

    # Add spheres to the mask at detected spot locations
    for z, y, x in zip(Z_coords, Y_coords, X_coords):
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d.shape)
        mask[sphere_mask] = 1

    return mask

# Load kinetochore coordinates for all timepoints using CuPy
with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]  # Shape (T, 1)

                # Loop through all timepoints
                for T_index in range(num_timepoints):
                    first_entry_ref = allCoordPix[T_index, 0]

                    if isinstance(first_entry_ref, h5py.Reference):
                        first_data = f[first_entry_ref]
                        array_data = cp.array(first_data[...])  # Load directly into CuPy
                        print(f"T={T_index}: Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

                        # Extract ZYX coordinates and convert MATLAB (1-based) to Python (0-based) indexing
                        Z_coords = cp.round(array_data[2] - 1).astype(cp.int32)  # Z values
                        Y_coords = cp.round(array_data[1] - 1).astype(cp.int32)  # Y values
                        X_coords = cp.round(array_data[0] - 1).astype(cp.int32)  # X values

                        # Ensure coordinates are within bounds
                        valid_mask = (Z_coords >= 0) & (Z_coords < num_z) & \
                                     (Y_coords >= 0) & (Y_coords < num_y) & \
                                     (X_coords >= 0) & (X_coords < num_x)
                        Z_coords, Y_coords, X_coords = Z_coords[valid_mask], Y_coords[valid_mask], X_coords[valid_mask]

                        # Generate mask using CuPy
                        kinetochore_volume[T_index] = generate_mask(kinetochore_volume[T_index], Z_coords, Y_coords, X_coords)

# Transfer CuPy array back to NumPy for visualization in Napari
kinetochore_volume_np = cp.asnumpy(kinetochore_volume)

# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add the entire image time series (T, Z, C, Y, X)
viewer.add_image(image_data[:, :, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='additive')
viewer.add_image(image_data[:, :, 1], colormap='cyan', scale=spacing, name='Channel 2', rendering='additive')

# Add kinetochore spheres as a 4D volume
viewer.add_labels(kinetochore_volume_np, colormap='yellow', scale=spacing, name="Kinetochores", opacity=0.8)

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles for 3D view

# Run Napari
napari.run()


T=0: Extracted array_data shape: (6, 125), Type: float64
T=1: Extracted array_data shape: (6, 59), Type: float64


## Use this one

In [ ]:
import tifffile
data = tifffile.imread(r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif")

In [3]:
import h5py
import numpy as np

file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"

with h5py.File(file_path, 'r') as f:
    # Dereference the first entry in dataStruct
    data_struct = f["dataStruct"][0, 0]  # Assuming first struct is relevant
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Dereference dataStruct
        
        # Navigate to initCoord
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            # Check if allCoordPix exists
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]
                
                # Read the first entry
                first_entry_ref = allCoordPix[40, 0]  # Assuming shape (127,1)
                
                if isinstance(first_entry_ref, h5py.Reference):
                    first_data = f[first_entry_ref]  # Dereference
                    print(f"First entry shape: {first_data.shape}, Type: {first_data.dtype}")
                    
                    # Load into a NumPy array
                    array_data = first_data[...]
                    print("First entry array:", array_data)
                else:
                    print("First entry is not a reference.")
            else:
                print("allCoordPix not found in initCoord.")
        


First entry shape: (6, 80), Type: float64
First entry array: [[1.34960881e+02 1.55093485e+02 1.36708452e+02 1.21191147e+02
  1.69424211e+02 1.19061538e+02 1.63067673e+02 1.30501061e+02
  1.41384288e+02 1.73764484e+02 1.60033560e+02 1.29842712e+02
  1.51080964e+02 1.60610009e+02 1.27493171e+02 1.04850252e+02
  1.61826521e+02 1.54792151e+02 1.47254149e+02 1.22453061e+02
  1.62688666e+02 1.52712275e+02 1.18902545e+02 1.00593792e+02
  1.36280828e+02 1.44088539e+02 1.41283013e+02 1.48143459e+02
  1.24475584e+02 1.74181332e+02 9.62688667e+01 1.59040597e+02
  1.43855369e+02 1.41078916e+02 1.14456612e+02 1.62011528e+02
  1.47916207e+02 1.09320316e+02 1.12435571e+02 1.67594002e+02
  1.53050876e+02 1.70891432e+02 1.08171164e+02 1.05017496e+02
  1.67177757e+02 9.99671086e+01 1.41017282e+02 1.32063578e+02
  8.50056133e+01 1.30459260e+02 1.52103305e+02 1.40083221e+02
  1.24628095e+02 1.69943342e+02 1.17157924e+02 1.58693400e+02
  1.25235371e+02 1.37475197e+02 1.62336385e+02 1.25875072e+02
  1.49649

In [2]:
import h5py
import cupy as cp
import napari
import tifffile

# File paths
mat_file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"
tiff_path = r"Z:\Shared299\KinetochoreClub\for Nina\03 PhD lattice movies\20241107 NP_002 MC256 2-colour 4-sec NEBD\Decon 10 iter with day-old PSFs\Cell033_Slide 6 capture 8_DC_DS.tif"

# Load the TIFF image (shape: T, Z, C, Y, X)
full_image_data = tifffile.imread(tiff_path)

image_data = full_image_data.copy()
num_timepoints, num_z, _, num_y, num_x = image_data.shape  # Extract image dimensions

# Voxel spacing for correct 3D rendering
z_spacing, y_spacing, x_spacing = 0.271, 0.104, 0.104
spacing = [z_spacing, y_spacing, x_spacing]

# Initialize an empty CuPy volume for kinetochores (T, Z, Y, X)
kinetochore_volume = cp.zeros((num_timepoints, num_z, num_y, num_x), dtype=cp.uint8)

# Define kinetochore sphere radius
sphere_radius = 2  # Adjust as needed

# Function to create a sphere using CuPy
def create_sphere(center, radius, shape):
    """Generates a binary sphere in a 3D CuPy array at the specified center."""
    z, y, x = cp.indices(shape, dtype=cp.float32)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image using CuPy
def generate_mask(image_3d, Z_coords, Y_coords, X_coords, spot_radius=2):
    """Generates a binary mask with spheres at kinetochore locations using CuPy."""
    
    # Create an empty CuPy mask
    mask = cp.zeros_like(image_3d, dtype=cp.uint8)

    # Add spheres to the mask at detected spot locations
    for z, y, x in zip(Z_coords, Y_coords, X_coords):
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d.shape)
        mask[sphere_mask] = 1

    return mask

# Load kinetochore coordinates for all timepoints using CuPy
with h5py.File(mat_file_path, 'r') as f:
    data_struct = f["dataStruct"][0, 0]  # Dereference first struct
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Get actual struct group
        
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]  # Shape (T, 1)

                # Loop through all timepoints
                for T_index in range(num_timepoints):
                    first_entry_ref = allCoordPix[T_index, 0]

                    if isinstance(first_entry_ref, h5py.Reference):
                        first_data = f[first_entry_ref]
                        array_data = cp.array(first_data[...])  # Load directly into CuPy
                        print(f"T={T_index}: Extracted array_data shape: {array_data.shape}, Type: {array_data.dtype}")

                        # Extract ZYX coordinates and convert MATLAB (1-based) to Python (0-based) indexing
                        Z_coords = cp.round(array_data[2] - 1).astype(cp.int32)  # Z values
                        Y_coords = cp.round(array_data[0] - 1).astype(cp.int32)  # Y values
                        X_coords = cp.round(array_data[1] - 1).astype(cp.int32)  # X values

                        # Ensure coordinates are within bounds
                        valid_mask = (Z_coords >= 0) & (Z_coords < num_z) & \
                                     (Y_coords >= 0) & (Y_coords < num_y) & \
                                     (X_coords >= 0) & (X_coords < num_x)
                        Z_coords, Y_coords, X_coords = Z_coords[valid_mask], Y_coords[valid_mask], X_coords[valid_mask]

                        # Generate mask using CuPy
                        kinetochore_volume[T_index] = generate_mask(kinetochore_volume[T_index], Z_coords, Y_coords, X_coords)

# Transfer CuPy array back to NumPy for visualization in Napari
kinetochore_volume_np = cp.asnumpy(kinetochore_volume)

# Create Napari viewer
viewer = napari.Viewer(ndisplay=3)  # Open in 3D mode

# Add the entire image time series (T, Z, C, Y, X)
viewer.add_image(image_data[:, :, 0], colormap='magenta', scale=spacing, name='Channel 1', rendering='attenuated_mip', opacity=0.5)
viewer.add_image(image_data[:, :, 1], colormap='cyan', scale=spacing, name='Channel 2', rendering='attenuated_mip', opacity=0.5)

# Add kinetochore spheres as a 4D volume
viewer.add_labels(kinetochore_volume_np, scale=spacing, name="Kinetochores", opacity=0.8)

# Set the camera view to a good starting angle
viewer.camera.angles = (45, 45, 90)  # Adjust angles for 3D view

# Run Napari
napari.run()


T=0: Extracted array_data shape: (6, 125), Type: float64
T=1: Extracted array_data shape: (6, 59), Type: float64
T=2: Extracted array_data shape: (6, 86), Type: float64
T=3: Extracted array_data shape: (6, 89), Type: float64
T=4: Extracted array_data shape: (6, 94), Type: float64
T=5: Extracted array_data shape: (6, 76), Type: float64
T=6: Extracted array_data shape: (6, 127), Type: float64
T=7: Extracted array_data shape: (6, 70), Type: float64
T=8: Extracted array_data shape: (6, 86), Type: float64
T=9: Extracted array_data shape: (6, 68), Type: float64
T=10: Extracted array_data shape: (6, 90), Type: float64
T=11: Extracted array_data shape: (6, 98), Type: float64
T=12: Extracted array_data shape: (6, 93), Type: float64
T=13: Extracted array_data shape: (6, 81), Type: float64
T=14: Extracted array_data shape: (6, 118), Type: float64
T=15: Extracted array_data shape: (6, 110), Type: float64
T=16: Extracted array_data shape: (6, 65), Type: float64
T=17: Extracted array_data shape: (6,

In [8]:
import os
import re
import tifffile as tiff
import numpy as np
from collections import defaultdict

def extract_position_and_time(filename):
    """
    Extracts position and timepoint from the filename.
    Assumes the format: 'Capture 1 - Position X [Y].Project Maximum Z_XY..._Z0_TTTT_C0.tif'
    """
    position_match = re.search(r'Position (\d+)', filename)
    time_match = re.search(r'_T(\d+)_C0\.tif', filename)

    if position_match and time_match:
        position = int(position_match.group(1))  # Extract position number
        timepoint = int(time_match.group(1))  # Extract timepoint number
        return position, timepoint
    return None, None

def process_timelapse(folder_path, output_prefix):
    """
    Processes all TIFF images in the folder, grouping them by position and timepoints,
    and saves each position's time series as a composite TIFF stack.
    """
    file_dict = defaultdict(list)  # Dictionary to store file paths per position

    # Scan folder and organize files by position
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".tif"):
            position, timepoint = extract_position_and_time(filename)
            if position is not None and timepoint is not None:
                file_dict[position].append((timepoint, os.path.join(folder_path, filename)))

    # Process each position
    for position in sorted(file_dict.keys()):
        # Sort files by timepoint
        sorted_files = sorted(file_dict[position], key=lambda x: x[0])
        image_stack = []

        # Read images and append to stack
        for _, file_path in sorted_files:
            img = tiff.imread(file_path)
            image_stack.append(img)

        # Convert to numpy array for saving
        if image_stack:
            output_filename = os.path.join(folder_path, f"{output_prefix}P{position}.tif")
            tiff.imwrite(output_filename, np.array(image_stack), photometric='minisblack')

        print(f"Saved {output_filename} with {len(image_stack)} timepoints.")

# Example usage:

process_timelapse(folder_path, output_prefix)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/path/to/your/folder'

In [9]:
import os
import re
import tifffile as tiff
import numpy as np
from collections import defaultdict

def extract_position_and_time(filename):
    """
    Extracts position and timepoint from the filename.
    Assumes the format: 'Capture 1 - Position X [Y].Project Maximum Z_XY..._Z0_TTTT_C0.tif'
    """
    position_match = re.search(r'Position (\d+)', filename)
    time_match = re.search(r'_T(\d+)_C0\.tif', filename)

    if position_match and time_match:
        position = int(position_match.group(1))  # Extract position number
        timepoint = int(time_match.group(1))  # Extract timepoint number
        return position, timepoint
    return None, None

def process_timelapse(folder_path, output_prefix):
    """
    Processes all TIFF images in the folder, grouping them into 16 separate stacks,
    each corresponding to a different position (P1 to P16).
    """
    file_dict = defaultdict(list)  # Dictionary to store file paths per position

    # Scan folder and organize files by position
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".tif"):
            position, timepoint = extract_position_and_time(filename)
            if position is not None and timepoint is not None:
                file_dict[position].append((timepoint, os.path.join(folder_path, filename)))

    # Determine the total number of timepoints
    num_positions = 16  # Expected number of positions
    total_files = sum(len(files) for files in file_dict.values())
    num_timepoints = total_files // num_positions  # Corrected number of actual timepoints

    print(f"Detected {num_timepoints} timepoints across {num_positions} positions.")

    # Process each position
    for position in range(1, num_positions + 1):
        if position in file_dict:
            # Sort files by timepoint
            sorted_files = sorted(file_dict[position], key=lambda x: x[0])
            image_stack = []

            # Read images and append to stack
            for _, file_path in sorted_files:
                img = tiff.imread(file_path)
                image_stack.append(img)

            # Convert to numpy array for saving
            if image_stack:
                output_filename = os.path.join(folder_path, f"{output_prefix}P{position}.tif")
                tiff.imwrite(output_filename, np.array(image_stack), photometric='minisblack')

                print(f"Saved {output_filename} with {len(image_stack)} timepoints.")

# Example usage:
folder_path = r"Z:\Shared243\sbrooks\20xdata\20250317_mc225_c1_individual"
output_prefix = "20250317_mc225_c1_"
process_timelapse(folder_path, output_prefix)


Detected 49 timepoints across 16 positions.
Saved Z:\Shared243\sbrooks\20xdata\20250317_mc225_c1_individual\20250317_mc225_c1_P1.tif with 784 timepoints.
